In [35]:
#각종 패키지 설치

import pandas as pd
!pip install webdriver-manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

from urllib.request import urlopen
#from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
from selenium.webdriver.common.by import By


url = 'https://www.airport.kr/co/ko/cpr/statisticOfLocalAirport.do'
#'https://www.airport.kr/co/ko/cpr/statisticCategoryOfLocal.do'

In [608]:
# 빈 데이터프레임 생성 및 컬럼 설정
air_info = pd.DataFrame(columns = {'지역','국가','운항_도착','운항_출발','운항_합계','여객_도착','여객_출발','여객_합계','화물_도착','화물_출발','화물_합계','년','월'})
air_info.columns = [['지역','국가','운항_도착','운항_출발','운항_합계','여객_도착','여객_출발','여객_합계','화물_도착','화물_출발','화물_합계','년','월']]

start = time.time()  # 시작 시간 저장

# 항공기 선택창 Xpath
xpath_airplane = '//*[@id="ARPLN_SE"]'
# 여객 선택창
xpath_type = '//*[@id="PAS_SEL"]'
# 검색 버튼
xpath_search = '//*[@id="btnSearch"]'
# 시작 년도
xpath_start_year = '//*[@id="S_YEAR"]'
# 시작 월
xpath_start_month = '//*[@id="S_MONTH"]'
# 기준년도
xpath_end_year = '//*[@id="E_YEAR"]'
# 기준월
xpath_end_month = '//*[@id="E_MONTH"]'



# 제공받은 주소 (인천국제공항공사)
url = 'https://www.airport.kr/co/ko/cpr/statisticCategoryOfLocal.do'


# 웹드라이버 열기
driver = webdriver.Chrome(r"C:\Users\USER\Downloads\chromedriver.exe")
driver.get(url)

# 항공기(여객기) 선택
driver.find_element(by=By.XPATH, value = xpath_airplane).click()
driver.find_element(by=By.XPATH, value = xpath_airplane).send_keys(Keys.DOWN)
driver.find_element(by=By.XPATH, value = xpath_airplane).send_keys(Keys.RETURN)

# 여객 (유임승객) 선택
driver.find_element(by=By.XPATH, value = xpath_type).click()
driver.find_element(by=By.XPATH, value = xpath_type).send_keys(Keys.DOWN)
driver.find_element(by=By.XPATH, value = xpath_type).send_keys(Keys.RETURN)

# 검색하고자하는 날짜 지정
for i in range(2013, 2022) : #13년도부터 21년도까지의 데이터 수집을 위해 반복문 설정
    driver.find_element(by=By.XPATH, value = xpath_start_year).send_keys(i) #시작년 체크
    driver.find_element(by=By.XPATH, value = xpath_end_year).send_keys(i) #종료년 체크
    for j in range(1,13) :
        if i == 2021 and j == 4 :
            break
        if j < 10 :
            jj = "0" + str(j)
        else :
            jj = j
        driver.find_element(by=By.XPATH, value = xpath_start_month).send_keys(jj) #시작월 체크
        driver.find_element(by=By.XPATH, value = xpath_end_month).send_keys(jj) #종료월 체크
        time.sleep(0.1) 
        
        # 검색 버튼 클릭
        driver.find_element(by=By.XPATH, value = xpath_search).click()
        
        #텍스트 파일 불러오기
        df = "".join(driver.find_element(by=By.XPATH, value = '/html/body/section[2]/div/div/article[1]/table/tbody').text).replace('\n',' {} {}\n'.format(i,j)).split("\n")
        a = pd.DataFrame(df[:-9]) # 합계와 전년대비 증감률은 형식이 다르기 때문에 가공 후 추후에 통합
        a.columns = ['col'] # 임시 컬럼명 설정
        a = pd.DataFrame(a.col.str.split(" ",12).tolist()) # 문자열 공백을 기준으로 데이터프레임 생성
        a.columns = [['지역','국가','운항_도착','운항_출발','운항_합계','여객_도착','여객_출발','여객_합계','화물_도착','화물_출발','화물_합계','년','월']]
        sub_info = ("".join(df[-9:]).replace("{} {}".format(str(i),j),"")).split() # 지정된 여러 개의 년-월 제거
        sub_info.append(i) # 년 데이터 추가
        sub_info.append(j) # 월 데이터 추가
        a = a.append(pd.Series(sub_info, index=a.columns), ignore_index=True) # 기존의 지역별 정보에 합계&전년대비 증감률 추가
        air_info = pd.concat([air_info,a],axis=0) # 최종 통합
#print(driver.current_url)

C:\Users\USER\AppData\Local\Temp/ipykernel_18212/3277370821.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\USER\Downloads\chromedriver.exe")


- 년, 월 구분을 위해 정수로 변경

In [ ]:
air_info['년'] = air_info['년'].astype(int)
air_info['월'] = air_info['월'].astype(int)

- 조건에 맞는 컬럼만 분리

In [632]:
tour_air_info = air_info[['지역','국가','여객_도착','여객_출발','여객_합계',"년","월"]]

In [636]:
tour_air_info.to_excel('airport_info.xlsx')

In [3]:
aa = pd.read_excel("인천공항_지역별통계.xlsx")

In [28]:
aa = aa[aa['지역'] != "합"]
aa = aa[aa['지역'] != "전년대비"]
aa.iloc[1:,1:].to_excel("공항통계_0421.xlsx", index=False)